In [4]:
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras 
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow
import tarfile
import cv2
import h5py
import imageio
from PIL import Image
import matplotlib.pyplot as plt
from PIL import ImageOps
import pandas as pd
from skimage.util import img_as_ubyte
import csv
from numpy import savez_compressed
from numpy import load
import glob
import zipfile
%matplotlib inline
#from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation,Input
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Convolution2D
from keras.layers.core import Dropout
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical,plot_model
from keras.utils.vis_utils import model_to_dot
import pydot
import graphviz
import pydotplus

In [5]:
import os
os.environ["PATH"] += '/home/ecbm4040/miniconda3/envs/envTF113/lib/python3.6/site-packages/'

In [6]:
# read train image infomation from training array, which is stored in train_cropped.npz
# tra_images has shape (33402,64,64,3)
train_data1 = load(os.getcwd()+'/data/train_cropped.npz', allow_pickle=True)
train_images = np.asarray(train_data1['arr_0'])
print(train_images.shape)
# read array infomation from training array, which is stored in train_cropped.npz
# tra_images has shape (33402,6)
label_data1 = load(os.getcwd()+'/data/train_label.npz', allow_pickle=True)
train_labels = np.asarray(label_data1['arr_0'])
print(train_labels.shape)

(33402, 64, 64, 3)
(33402, 6)


In [7]:
test_data1 = load(os.getcwd()+'/data/test_cropped.npz', allow_pickle=True)
test_images = np.asarray(test_data1['arr_0'])
print(test_images.shape)
label_data2 = load(os.getcwd()+'/data/label/test_label.npz', allow_pickle=True)
test_labels = np.asarray(label_data2['arr_0'])
print(test_labels.shape)

(13068, 64, 64, 3)
(13068, 6)


In [8]:
x_train = train_images
x_test = test_images

In [10]:
def change_shape(array1):
    label_emp = np.empty((array1.shape[1],array1.shape[0]), dtype='uint8')
    for i in range(array1.shape[0]):
        for j in range(array1.shape[1]):
            label_emp[j,i] = array1[i,j]
    return label_emp

In [11]:
y_train = change_shape(train_labels)
y_test = change_shape(test_labels)
if y_train.shape[0] == 6:
    print('y_train shape is right')
if y_test.shape[0] == 6:
    print('y_train shape is right')
# print(y_train[0].reshape([-1,1]))
# print(y_train.shape)

y_train shape is right
y_train shape is right


In [13]:
def digit_to_categorical_label(data):
    n = data.shape[0]   
    data_cat = np.empty([n, 6])
    for i in range(n):
        data_cat[i] = to_categorical(data[i], num_classes=6)        
    return data_cat

In [14]:
def digit_to_categorical(data):
    n = data.shape[1]
    data_cat = np.empty([len(data), n, 11])    
    for i in range(n):
        data_cat[:, i] = to_categorical(data[:, i], num_classes=11)        
    return data_cat


In [15]:
y_train_num = digit_to_categorical_label(y_train[0])
y_train_digits = digit_to_categorical(y_train[1:])
y_test_num = digit_to_categorical_label(y_test[0])
y_test_digits = digit_to_categorical(y_test[1:])
print(y_train_num.shape)
print(y_train_digits.shape)
print(y_test_num.shape)
print(y_test_digits.shape)

(33402, 6)
(5, 33402, 11)
(13068, 6)
(5, 13068, 11)


In [16]:
def build_model():
    identifier = Sequential()
    Input_1 = Input(shape=(64,64,3))
    conv_model = BatchNormalization()(Input_1)
    conv_model = Convolution2D(48, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(54, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(1,1))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(128, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(conv_model)
    conv_model = Dropout(0.1)(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(160, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(1,1))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(192, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(192, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(1,1))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(192, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(2,2))(conv_model)

    conv_model = BatchNormalization()(conv_model)
    conv_model = Convolution2D(192, (5, 5), activation='relu',padding="same")(conv_model)
    conv_model = MaxPooling2D(pool_size=(2, 2), strides=(1,1))(conv_model)

    conv_model = Flatten()(conv_model)
    conv_model = Dense(1024,activation = 'relu')(conv_model)
    conv_model = Dense(512,activation = 'relu')(conv_model)

    x1 = Dense(6, activation = 'softmax')(conv_model)
    x2 = Dense(11, activation = 'softmax')(conv_model)
    x3 = Dense(11, activation = 'softmax')(conv_model)
    x4 = Dense(11, activation = 'softmax')(conv_model)
    x5 = Dense(11, activation = 'softmax')(conv_model)
    x6 = Dense(11, activation = 'softmax')(conv_model)

    y = [x1,x2,x3,x4,x5,x6]

    identifier = Model(inputs = Input_1, outputs = y)
    return identifier


In [ ]:
# cross validation
from sklearn.model_selection import KFold
 
n_split=10
 
for train_index,test_index in KFold(n_split).split(x_train):
    x_train_split,x_test_split=x_train[train_index],x_train[test_index]
    y_train_num_split,y_test_num_split =y_train_num[train_index],y_train_num[test_index]
    y_train_digits0,y_test_digits0 = y_train_digits[0][train_index],y_train_digits[0][test_index]
    y_train_digits1,y_test_digits1 = y_train_digits[1][train_index],y_train_digits[1][test_index]
    y_train_digits2,y_test_digits2 = y_train_digits[2][train_index],y_train_digits[2][test_index]
    y_train_digits3,y_test_digits3 = y_train_digits[3][train_index],y_train_digits[3][test_index]
    y_train_digits4,y_test_digits4 = y_train_digits[4][train_index],y_train_digits[4][test_index]
    
    
    model_cv= build_model()
    model_cv.compile(optimizer = "adam", loss = "categorical_crossentropy",metrics = ["accuracy"])
    #model_cv.fit(x=x_train,
    #           y=[y_train_num,y_train_digits[0],y_train_digits[1],y_train_digits[2],y_train_digits[3],y_train_digits[4]],
     #           batch_size=64, epochs=20,verbose=1,validation_split=0.2,shuffle=True)
    model_cv.fit(x=x_train_split,y=[y_train_num_split,y_train_digits0,y_train_digits1,y_train_digits2,y_train_digits3,y_train_digits4], batch_size=64,shuffle=True)
    
    print('Model evaluation ',model_cv.evaluate(x=x_test_split,
               y=[y_test_num_split,y_test_digits0,y_test_digits1,y_test_digits2,y_test_digits3,y_test_digits4]))


In [17]:
import skopt
from skopt import gbrt_minimize, gp_minimize

/home/ecbm4040/miniconda3/envs/envTF113/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [18]:
from keras.optimizers import Adam

In [19]:
from skopt.utils import use_named_args

In [20]:
# function for get results of model for plotting
def get_array(his_name,i):
    val_loss = np.asarray(his_name.history['val_loss'])
    val_acc1 = np.asarray(his_name.history[f'val_dense_{i}_accuracy'])
    val_acc2 = np.asarray(his_name.history[f'val_dense_{i+1}_accuracy'])
    val_acc3 = np.asarray(his_name.history[f'val_dense_{i+2}_accuracy'])
    val_acc4 = np.asarray(his_name.history[f'val_dense_{i+3}_accuracy'])
    val_acc5 = np.asarray(his_name.history[f'val_dense_{i+4}_accuracy'])
    val_acc6 = np.asarray(his_name.history[f'val_dense_{i+5}_accuracy'])
    val_acc = val_acc1*val_acc2*val_acc3*val_acc4*val_acc5*val_acc6
    train_loss = np.asarray(his_name.history['loss'])
    train_acc1 = np.asarray(his_name.history[f'dense_{i}_accuracy'])
    train_acc2 = np.asarray(his_name.history[f'dense_{i+1}_accuracy'])
    train_acc3 = np.asarray(his_name.history[f'dense_{i+2}_accuracy'])
    train_acc4 = np.asarray(his_name.history[f'dense_{i+3}_accuracy'])
    train_acc5 = np.asarray(his_name.history[f'dense_{i+4}_accuracy'])
    train_acc6 = np.asarray(his_name.history[f'dense_{i+5}_accuracy'])
    train_acc =  train_acc1*train_acc2*train_acc3*train_acc4*train_acc5*train_acc6
    return val_loss,val_acc,train_loss,train_acc

In [21]:
from skopt.space import Real, Categorical, Integer  
dim_learning_rate = Real(low=1e-4, high=1e-2, prior='log-uniform',
                         name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_input_nodes = Integer(low=1, high=512, name='num_input_nodes')
dim_num_dense_nodes = Integer(low=1, high=28, name='num_dense_nodes')
dim_activation = Categorical(categories=['relu', 'sigmoid'],
                             name='activation')
dim_batch_size = Integer(low=1, high=128, name='batch_size')
dim_adam_decay = Real(low=1e-6,high=1e-2,name="adam_decay")

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_input_nodes,
              dim_num_dense_nodes,
              dim_activation,
              dim_batch_size,
              dim_adam_decay
             ]
default_parameters = [1e-3, 1,512, 13, 'relu',64, 1e-3]

In [22]:
from tensorflow.python.keras import backend as K

In [23]:
def set_up_model(learning_rate, adam_decay):
    #setup our optimizer and compile
    model = build_model()
    adam = Adam(lr=learning_rate, decay= adam_decay)
    model.compile(optimizer=adam, loss='categorical_crossentropy',
                     metrics=['accuracy'])
    return model

In [24]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_input_nodes, 
            num_dense_nodes,activation, batch_size,adam_decay):

    model = set_up_model(learning_rate,adam_decay)
    
    #named blackbox becuase it represents the structure
    fitted_model = model.fit(x=x_train,
                        y=[y_train_num,y_train_digits[0],y_train_digits[1],y_train_digits[2],y_train_digits[3],y_train_digits[4]],
                        epochs=15,
                        batch_size=batch_size,
                        validation_split=0.15,
                        )
    #return the validation accuracy for the last epoch.
    #print(model.metrics_names)
    res = [i for i in model.metrics_names[1] if i.isdigit()] 
    num = ''.join(res)
    accuracy = get_array(fitted_model,int(num))[1][0]
    #accuracy = fitted_model.history['val_acc'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy=" + str(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    tensorflow.reset_default_graph()
    
    # the optimizer aims for the lowest score, so we return our negative accuracy
    return -accuracy

In [ ]:
gp_result = gp_minimize(func=fitness,
                            dimensions=dimensions,
                            n_calls=12,
                            noise= 0.01,
                            n_jobs=-1,
                            kappa = 5,
                            x0=default_parameters)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 28391 samples, validate on 5011 samples
Epoch 1/15
28391/28391 [==============================] - 69s 2ms/step - loss: 7.2456 - dense_3_loss: 1.1325 - dense_4_loss: 2.0539 - dense_5_loss: 2.4034 - dense_6_loss: 1.3517 - dense_7_loss: 0.2956 - dense_8_loss: 0.0085 - dense_3_accuracy: 0.5370 - dense_4_accuracy: 0.2652 - dense_5_accuracy: 0.1435 - dense_6_accuracy: 0.6959 - dense_7_accuracy: 0.9555 - dense_8_accuracy: 0.9979 - val_loss: 7.2829 - val_dense_3_loss: 1.1232 - val_dense_4_loss: 2.0503 - val_dense_5_loss: 2.3940 - val_dense_6_loss: 1.3639 - val_dense_7_loss: 0.3366 - val_dense_8_loss: 0.0058 - val_dense_3_accuracy: 0.5468 - val_dense_4_accuracy: 0.2702 - val_dense_5_accuracy: 0.1453 - val_dense_6_accuracy: 0.6921 - val_dense_7_accuracy: 0.9513 - val_dense_8_accuracy: 0.9996
Epoch 2/15
28391/28391 [==============================] - 65s 2ms/step - loss: